In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo, fci

I build this scrit to see wether I can reproduce the electron electron interaction energy form the 2RDM, this serves as test to see wether I understand the data struvctures and theorie accurately to move on to other ideas.

The following function caculates the electron electron interactio  energy from the Fock basis representation of the 2RDM $\gamma_2$ in the follwong way:
$$E_{ee} = \sum_p \sum_q \sum_r \sum_s \gamma_{2_{pqrs}} \langle p q | r s \rangle$$

In [2]:
def ee_from_gamma_2(eri,dm2):
    E = 0 
    dim = eri.shape[0]
    for p in range(0,dim):
        for q in range(0,dim):
            for r in range(0,dim):
                for s in range(0,dim):
                    E+=eri[p,q,r,s]*dm2[p,q,r,s]
    return E

In [3]:
mol = gto.Mole()
mol.atom = """
    He    0.    0.    0.
"""
#mol.basis = "cc-pvdz"
mol.basis = "6-31g" 
mol.build()
print(mol.nelec)

(1, 1)


In [24]:
# Run Hartree-Fock.
mf = scf.RHF(mol)
mf.kernel()
hf_1RDM = mf.make_rdm1() # -> AO basis
hf_2RDM = mf.make_rdm2() # -> AO basis
J = mf.get_j()
K = mf.get_k()
h = mf.get_hcore()
C = mf.mo_coeff
#P=np.matmul(C[:,0:N],C[:,0:N].T)
h1 = C.T@h@C
hf_1RDM_MO = C.T@hf_1RDM@C

converged SCF energy = -2.85516042615445


In [25]:
# Find electron-repulsion integrals (eri).
eri = ao2mo.kernel(mol, mf.mo_coeff)
eri = np.asarray(ao2mo.restore(1, eri, mol.nao))

In [26]:
print(eri.shape)

(2, 2, 2, 2)


In [27]:
# First create FCI solver with function fci.FCI and solve the FCI problem
#
cisolver = fci.FCI(mol, mf.mo_coeff)
e, fcivec = cisolver.kernel()

In [28]:
dm1, dm2 = cisolver.make_rdm12(fcivec, mf.mo_coeff.shape[1], mol.nelec)

In [29]:
print(.5*ee_from_gamma_2(eri,dm2), e-np.trace(np.matmul(dm1,h1)))

0.9934201474038266 0.9934201474038242


Reference energy from 1 rdm for Hartree Fock just to be sure what we are dealing with

In [32]:
print(np.trace(np.matmul(hf_1RDM,h))+1/2.*np.trace(np.matmul(hf_1RDM, J))+-1/4.*np.trace(np.matmul(hf_1RDM, K)))

-2.8551604261544465


In [33]:
print(np.trace(np.matmul(hf_1RDM,h)))
print(np.trace(np.matmul(dm1,h1)))


-3.882067595029823
-3.8635822863046467


In [11]:
print(hf_2RDM)

[[[[0.24578494 0.21319997]
   [0.21319997 0.18493495]]

  [[0.21319997 0.18493495]
   [0.18493495 0.16041718]]]


 [[[0.21319997 0.18493495]
   [0.18493495 0.16041718]]

  [[0.18493495 0.16041718]
   [0.16041718 0.13914985]]]]


In [12]:
print(.5*ee_from_gamma_2(eri,hf_2RDM), e-np.trace(np.matmul(dm1,h1)))

0.20548423672055594 0.9934201474038242
